In [2]:
import mpl_toolkits
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import pandas as pd
import matplotlib
import collections
import ast
#install basemap
from mpl_toolkits.basemap import Basemap
from load_yelp_data import load_yelp_dataframe
import graphlab
%matplotlib inline

A newer version of GraphLab Create (v2.1) is available! Your current version is v1.4.0.
You can use pip to upgrade the graphlab-create package. For more information see https://dato.com/products/create/upgrade.

#### As a baseline for collaborative filtering, we'll train a popularity recommender that just gives every user the most popular recommendations within the category.

In [3]:
businesses = load_yelp_dataframe('businesses').drop('Unnamed: 0', 1)
reviews = load_yelp_dataframe('reviews').drop('Unnamed: 0', 1)
users = load_yelp_dataframe('users').drop('Unnamed: 0', 1)

In [4]:
reviews.head()

,user_id,business_id,year,month,day,stars
0,511457,0,2012,8,1,4
1,291005,0,2014,2,13,5
2,551917,0,2015,10,31,5
3,307072,0,2015,12,26,3
4,658212,0,2016,4,8,2


In [5]:
# Classify the datapoints into the cities based on the latitude and the longitude
cities = np.array([['Edinburgh', 55.9533, -3.1883, 0, 0],
         ['Karlsruhe', 49.0069, 8.4037, 0, 0],
         ['Montreal', 45.5017, -73.5673, 0, 0],
         ['Waterloo', 43.4643, -80.5204, 0, 0],
         ['Pittsburgh', 40.4406, -79.9959, 0, 0],
         ['Charlotte', 35.2271, -80.8431, 0, 0],
         ['Urbana-Champaign', 40.1106, -88.2073, 0, -150000],
         ['Phoenix', 33.4484, -112.0740, 0, 0],
         ['Las Vegas', 36.1699, -115.1398, 0, 0],
         ['Madison', 43.0731, -89.4012, 0, 0]])
lat = np.array(cities[:,1], np.float32)
lon = np.array(cities[:,2], np.float32)
x_offsets = np.array(cities[:,3], np.float32)
y_offsets = np.array(cities[:,4], np.float32)

def classify_datapoint_by_latitude_and_longitude(datapoint):
#     print datapoint
    latitude = datapoint['latitude']
    longitude = datapoint['longitude']
    distances = (latitude - lat)**2 + (longitude - lon)**2
    min_dist = np.argmin(distances)
    return cities[min_dist, 0]

businesses['location'] = businesses.apply(classify_datapoint_by_latitude_and_longitude, axis=1)

In [6]:
reviews_w_biz = pd.merge(reviews, businesses, how='inner', on='business_id',
         suffixes=('_x', '_y'), copy=True, indicator=False)

<font color='blue'>Restrict to restaurants within the two areas we're thinking about. (Of course, people may also want recommendations in cities they're visiting, but we can deal with that later.)</font>

In [16]:
lv_reviews = reviews_w_biz[reviews_w_biz['location']=='Las Vegas']
reviews_lv_rest = lv_reviews[lv_reviews['categories'].apply( lambda x: 'Restaurants' in x)]
phx_reviews = reviews_w_biz[reviews_w_biz['location']=='Phoenix']
reviews_phx_rest = phx_reviews[phx_reviews['categories'].apply( lambda x: 'Restaurants' in x)]

In [8]:
lv_reviews.head()

,user_id,business_id,year,month,day,stars_x,attributes,categories,city,full_address,...,latitude,longitude,name,neighborhoods,open,review_count,stars_y,state,type,location
201184,420444,8411,2015,1,12,4,{},[],Blue Diamond,"8000 Blue Diamond Rd\nSouthwest\nBlue Diamond,...",...,36.020636,-115.299037,Spring Mountain Ranch,['Southwest'],True,3,4.5,NV,business,Las Vegas
201185,277412,8411,2015,7,21,5,{},[],Blue Diamond,"8000 Blue Diamond Rd\nSouthwest\nBlue Diamond,...",...,36.020636,-115.299037,Spring Mountain Ranch,['Southwest'],True,3,4.5,NV,business,Las Vegas
201186,20255,8411,2016,5,23,5,{},[],Blue Diamond,"8000 Blue Diamond Rd\nSouthwest\nBlue Diamond,...",...,36.020636,-115.299037,Spring Mountain Ranch,['Southwest'],True,3,4.5,NV,business,Las Vegas
201187,67269,8412,2007,4,7,4,"{'Accepts Credit Cards': True, 'Price Range': ...","['Food', 'Grocery', 'Shopping', 'Drugstores']",Henderson,"1000 N Green Valley Pkwy\nHenderson, NV 89012",...,36.027902,-115.084131,Smith's Food & Drug Stores,[],True,28,3.5,NV,business,Las Vegas
201188,23731,8412,2007,11,27,3,"{'Accepts Credit Cards': True, 'Price Range': ...","['Food', 'Grocery', 'Shopping', 'Drugstores']",Henderson,"1000 N Green Valley Pkwy\nHenderson, NV 89012",...,36.027902,-115.084131,Smith's Food & Drug Stores,[],True,28,3.5,NV,business,Las Vegas


<font color='blue'>Restrict to restaurants that have at least 11 reviews, then filter to only the columns we need to train the recommender.</font>

In [17]:
reviews_lv_rest = reviews_lv_rest[reviews_lv_rest['review_count']>10][['user_id','business_id','stars_x']]
reviews_phx_rest = reviews_phx_rest[reviews_phx_rest['review_count']>10][['user_id','business_id','stars_x']]

In [18]:
reviews_lv_rest.head()

,user_id,business_id,stars_x
201215,67269,8413,1
201216,143364,8413,3
201217,426091,8413,1
201218,505574,8413,1
201219,169826,8413,1


<font color='blue'>Use the graphlab package to train a popularity recommender and give reviews for the first 5 user_ids.</font>

In [19]:
train_data_lv = graphlab.SFrame(reviews_lv_rest)
train_data_phx = graphlab.SFrame(reviews_phx_rest)
# test_data = graphlab.SFrame(ratings_test)

In [20]:
popularity_model_lv = graphlab.popularity_recommender.create(train_data_lv, user_id='user_id', item_id='business_id', target='stars_x')

PROGRESS: Recsys training: model = popularity
PROGRESS: Preparing data set.
PROGRESS:     Data has 656088 observations with 224014 users and 4582 items.
PROGRESS:     Data prepared in: 1.19646s
PROGRESS: 656088 observations to process; with 4582 unique items.
PROGRESS: 656088 observations processed.
PROGRESS: Number observations / second: 7.23536e+06
PROGRESS: Computing nearest neighbors model for item similarity queries.
PROGRESS: +------------+--------------+
PROGRESS: | Tree level | Elapsed Time |
PROGRESS: +------------+--------------+
PROGRESS: | 0          | 2.658ms      |
PROGRESS: | 1          | 5.173ms      |
PROGRESS: | 2          | 7.832ms      |
PROGRESS: | 3          | 9.296ms      |
PROGRESS: +------------+--------------+


In [ ]:
popularity_model_lv = graphlab.popularity_recommender.create(train_data_lv, user_id='user_id', item_id='business_id', target='stars_x')

In [21]:
popularity_recomm_lv = popularity_model_lv.recommend(users=range(1,6),k=5)
popularity_recomm_lv.print_rows(num_rows=25)

+---------+-------------+-------+------+
| user_id | business_id | score | rank |
+---------+-------------+-------+------+
|    1    |    83929    |  5.0  |  1   |
|    1    |    83406    |  5.0  |  2   |
|    1    |    81070    |  5.0  |  3   |
|    1    |    28456    |  5.0  |  4   |
|    1    |    65521    |  5.0  |  5   |
|    2    |    83929    |  5.0  |  1   |
|    2    |    83406    |  5.0  |  2   |
|    2    |    81070    |  5.0  |  3   |
|    2    |    28456    |  5.0  |  4   |
|    2    |    65521    |  5.0  |  5   |
|    3    |    83929    |  5.0  |  1   |
|    3    |    83406    |  5.0  |  2   |
|    3    |    81070    |  5.0  |  3   |
|    3    |    28456    |  5.0  |  4   |
|    3    |    65521    |  5.0  |  5   |
|    4    |    83929    |  5.0  |  1   |
|    4    |    83406    |  5.0  |  2   |
|    4    |    81070    |  5.0  |  3   |
|    4    |    28456    |  5.0  |  4   |
|    4    |    65521    |  5.0  |  5   |
|    5    |    83929    |  5.0  |  1   |
|    5    |    8

In [35]:
businesses[businesses['business_id'].isin([83929,83406,81070,28456,65521])]

,attributes,business_id,categories,city,full_address,hours,latitude,longitude,name,neighborhoods,open,review_count,stars,state,type,location
28456,"{'Accepts Credit Cards': True, 'Noise Level': ...",28456,"['Arts & Entertainment', 'Casinos', 'Asian Fus...",Las Vegas,The Venetian Resort Hotel Casino\n3355 Las Veg...,{},36.122084,-115.168032,The Paiza Club,['The Strip'],True,15,5.0,NV,business,Las Vegas
65521,"{'Accepts Credit Cards': True, 'By Appointment...",65521,"['Food', 'Food Delivery Services', 'Gluten-Fre...",Las Vegas,"Northwest\nLas Vegas, NV 89128",{},36.233048,-115.246776,Body Food Kitchen,['Northwest'],False,12,5.0,NV,business,Las Vegas
81070,"{'Accepts Credit Cards': True, 'Noise Level': ...",81070,"['Mexican', 'Food', 'Caterers', 'Event Plannin...",Las Vegas,"2901 N Rancho Dr\nLas Vegas, NV 89130",{},36.212461,-115.208646,Taco Naco,[],True,13,5.0,NV,business,Las Vegas
83406,"{'Accepts Credit Cards': True, 'Good for Kids'...",83406,"['Food', 'Food Delivery Services', 'American (...",Las Vegas,6475 W Charleston Blvd\nSte 110\nWestside\nLas...,"{'Monday': {'close': '18:00', 'open': '09:00'}...",36.158604,-115.236715,Modern Proteins,['Westside'],True,27,5.0,NV,business,Las Vegas
83929,"{'Accepts Credit Cards': True, 'Noise Level': ...",83929,"['Japanese', 'Restaurants']",Las Vegas,"7331 W. Lake Mead Blvd.\nSte 104\nLas Vegas, N...","{'Monday': {'close': '22:30', 'open': '18:00'}...",36.194769,-115.252472,Kame Omakase,[],True,22,5.0,NV,business,Las Vegas


<font color='blue'>As we'd expect, these restaurants all have five stars.</font>

#### Now train a similarity recommender based on the same table.

In [36]:
item_sim_model_lv = graphlab.item_similarity_recommender.create(train_data_lv, user_id='user_id', item_id='business_id', target='stars_x', similarity_type='pearson')

PROGRESS: Recsys training: model = item_similarity
PROGRESS: Preparing data set.
PROGRESS:     Data has 656088 observations with 224014 users and 4582 items.
PROGRESS:     Data prepared in: 1.20379s
PROGRESS: Computing item similarity statistics:
PROGRESS: +-----------------+-----------------+
PROGRESS: | Number of users | Elapsed Time    |
PROGRESS: +-----------------+-----------------+
PROGRESS: | 100000          | 0.126002        |
PROGRESS: | 200000          | 0.395273        |
PROGRESS: +-----------------+-----------------+
PROGRESS: Computing most similar items for 4582 items:
PROGRESS: +-----------------+-----------------+
PROGRESS: | Number of items | Elapsed Time    |
PROGRESS: +-----------------+-----------------+
PROGRESS: | 1000            | 0.664789        |
PROGRESS: | 2000            | 0.730435        |
PROGRESS: | 3000            | 0.804128        |
PROGRESS: | 4000            | 0.866143        |
PROGRESS: +-----------------+-----------------+
PROGRESS: Finished trainin

In [38]:
item_sim_recomm_lv = item_sim_model_lv.recommend(users=range(15,20),k=5)
item_sim_recomm_lv.print_rows(num_rows=25)

+---------+-------------+---------------+------+
| user_id | business_id |     score     | rank |
+---------+-------------+---------------+------+
|    15   |    84947    | 6.43993589744 |  1   |
|    15   |    64256    | 6.32027777778 |  2   |
|    15   |    76459    | 6.26024193548 |  3   |
|    15   |    81142    | 6.14167355372 |  4   |
|    15   |    81609    | 6.05025193798 |  5   |
|    16   |    83929    |      5.0      |  1   |
|    16   |    83406    |      5.0      |  2   |
|    16   |    81070    |      5.0      |  3   |
|    16   |    28456    |      5.0      |  4   |
|    16   |    65521    |      5.0      |  5   |
|    17   |    79082    | 5.98666370053 |  1   |
|    17   |    80914    | 5.84247517703 |  2   |
|    17   |    83327    | 5.83776324728 |  3   |
|    17   |    69270    | 5.80965152352 |  4   |
|    17   |    77164    |  5.7986328125 |  5   |
|    18   |    68832    | 5.50165701913 |  1   |
|    18   |    84859    | 5.49939585463 |  2   |
|    18   |    81586

<font color='blue'>As we can see, some users are now receiving different specialized recommendations. Some also still only have the baseline recommendations, and this is likely because they haven't rated enough things personally for the system to generate personalized recommendations for them. In the future, we could filter these people out of the dataset, but it also seems reasonable to just recommend to them the most highly-rated restaurants.</font>

In [39]:
businesses[businesses['business_id'].isin([84947,64256,76459,81142,81609])]

,attributes,business_id,categories,city,full_address,hours,latitude,longitude,name,neighborhoods,open,review_count,stars,state,type,location
64256,"{'Accepts Credit Cards': True, 'Noise Level': ...",64256,"['Comfort Food', 'Mexican', 'Restaurants']",Las Vegas,"240 N Jones Blvd\nSte E\nLas Vegas, NV 89107","{'Monday': {'close': '19:00', 'open': '09:00'}...",36.175601,-115.222265,Chicken Itza,[],False,13,4.5,NV,business,Las Vegas
76459,"{'Accepts Credit Cards': True, 'Noise Level': ...",76459,"['Bakeries', 'Food', 'American (Traditional)',...",Las Vegas,6070 S Rainbow Blvd\nSte 7\nSpring Valley\nLas...,"{'Monday': {'close': '18:30', 'open': '11:00'}...",36.078686,-115.241801,Pie Me Over,['Spring Valley'],True,70,4.5,NV,business,Las Vegas
81142,"{'Accepts Credit Cards': True, 'Noise Level': ...",81142,"['Pizza', 'Restaurants']",Las Vegas,5270 S Fort Apache Rd\nSte 320\nSpring Valley\...,"{'Monday': {'close': '21:00', 'open': '11:00'}...",36.093101,-115.296562,The Pizza Kitchen,['Spring Valley'],True,137,4.5,NV,business,Las Vegas
81609,"{'Accepts Credit Cards': True, 'Noise Level': ...",81609,"['Barbeque', 'Restaurants']",Las Vegas,"5611 S Valley View Blvd\nLas Vegas, NV 89118","{'Monday': {'close': '18:00', 'open': '11:30'}...",36.087861,-115.190509,Jessie Rae's BBQ,[],True,153,4.5,NV,business,Las Vegas
84947,"{'Accepts Credit Cards': True, 'Noise Level': ...",84947,"['Restaurants', 'Italian']",Henderson,"72 Horizon Ridge Pkwy\nSte 100\nHenderson, NV ...","{'Monday': {'close': '21:00', 'open': '11:00'}...",36.013193,-114.992711,Il Chianti Italian,[],True,51,5.0,NV,business,Las Vegas
